In [1]:
import os
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import skimage
from skimage.io import imread
from skimage.transform import resize
import glob
import pandas as pd
from tqdm import tqdm
import pickle

In [2]:
with open('Models/DR/trainedPCA.pkl', 'rb') as picklefile:
    pca=pickle.load(picklefile)
    
with open('Data/imputed_valid_data.pkl', 'rb') as picklefile:
    valid_data=pickle.load(picklefile)

with open('Data/imputed_train_data.pkl', 'rb') as picklefile:
    train_data=pickle.load(picklefile)    
    
with open('valid_dates.pkl', 'rb') as picklefile:
    valid_dates=pickle.load(picklefile)

with open('train_dates.pkl', 'rb') as picklefile:
    train_dates=pickle.load(picklefile)        

In [3]:
columns=[]
for i in range(1,1 + pca.n_components_):
    columns.append('unit'+str(i))

In [5]:
train_df=pd.DataFrame(columns=columns)
valid_df=pd.DataFrame(columns=columns)

In [6]:
train_df.insert(0, "date",[])
valid_df.insert(0, "date",[])

In [7]:
for date,img in tqdm(zip(train_dates,train_data)):
    encoded_img = pca.transform(img.reshape(1,256*272))
    encoded_img=np.reshape(encoded_img,(pca.n_components_))
    temp_dict={}
    temp_dict['date']= date
    count=0
    for i in encoded_img:
        temp_dict['unit'+str(count+1)]=i
        count=count+1    
    train_df = train_df.append(temp_dict,ignore_index=True)

1822it [02:13, 13.69it/s]


In [22]:
for date,img in tqdm(zip(valid_dates,valid_data)):
    encoded_img = pca.transform(img.reshape(1,256*272))
    encoded_img=np.reshape(encoded_img,(pca.n_components_))
    temp_dict={}
    temp_dict['date']= date
    count=0
    for i in encoded_img:
        temp_dict['unit'+str(count+1)]=i
        count=count+1    
    valid_df = valid_df.append(temp_dict,ignore_index=True)

139it [00:09, 14.21it/s]


In [23]:
train_df.to_csv(r'imputed_train_encoded_data.csv', index = False)

In [24]:
valid_df.to_csv(r'imputed_valid_encoded_data.csv', index = False)